In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key

# Import API key
gmaps.configure(api_key = g_key)

In [2]:
#connecting to weatherpy data
weather_data = "Global Weather Data.csv"
weather_df = pd.read_csv(weather_data)
weather_df.dropna(inplace = True) 
weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Windspeed
0,0,bengkulu,100.0,ID,1613395278,85.0,-3.8004,102.2655,79.50,3.29
1,1,camacha,40.0,PT,1613395348,72.0,33.0833,-16.3333,64.40,8.05
2,2,bluff,91.0,NZ,1613395348,81.0,-46.6000,168.3333,48.00,1.01
3,3,saint-philippe,90.0,RE,1613395348,61.0,-21.3585,55.7679,84.20,4.61
4,4,kavali,0.0,IN,1613395349,70.0,14.9167,79.9833,74.23,8.81
...,...,...,...,...,...,...,...,...,...,...
427,476,diglur,8.0,IN,1613395471,20.0,18.5500,77.6000,82.89,4.76
428,477,semporna,20.0,MY,1613395471,89.0,4.4818,118.6112,78.80,10.16
429,478,honiara,100.0,SB,1613395471,81.0,-9.4333,159.9500,79.20,2.44
430,479,majene,98.0,ID,1613395471,80.0,-3.5403,118.9707,78.98,3.62


In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill empty values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [4]:
# created gmap figure
fig = gmaps.figure()

In [5]:
# add heat layer to fig
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
#create new dataframe storing ideal weather criteria
Ideal_df = pd.DataFrame(weather_df, columns = ["City","Country","Max Temperature", "Windspeed", "Cloudiness", "Humidity", "Latitude", "Longitude"])
Ideal_df.head()
IdealTemp = (Ideal_df['Max Temperature'] < 80) & (Ideal_df["Max Temperature"] > 70) 
IdealWind = Ideal_df['Windspeed'] < 10
IdealCloudiness = Ideal_df['Cloudiness'] == 0 
IdealHumidity = Ideal_df['Humidity'] <= 60


In [18]:
#store into new df called hotel_df
hotel_df = Ideal_df[IdealTemp & IdealWind & IdealCloudiness & IdealHumidity]
hotel_df.reset_index(drop=True, inplace=True)
hotel_df


,City,Country,Max Temperature,Windspeed,Cloudiness,Humidity,Latitude,Longitude
0,umm lajj,SA,79.07,7.58,0.0,51.0,25.0213,37.2685
1,dezful,IR,78.57,4.79,0.0,22.0,32.3811,48.4058
2,chandrakona,IN,73.81,5.79,0.0,22.0,22.7333,87.5167
3,mailsi,PK,74.62,5.39,0.0,19.0,29.8003,72.1758
4,sur,OM,75.63,3.80,0.0,52.0,22.5667,59.5289
5,pyu,MM,73.76,3.98,0.0,28.0,18.4833,96.4333
6,bubaque,GW,77.49,3.56,0.0,50.0,11.2833,-15.8333
7,dehloran,IR,76.69,2.80,0.0,18.0,32.6941,47.2679
8,bheramara,BD,72.45,3.29,0.0,22.0,24.0232,88.9925
9,puerto del rosario,ES,71.60,6.91,0.0,46.0,28.5004,-13.8627


In [20]:
#establishing variables for query loop
hotel_list = []
radius = 5000
search = "lodging"
query_type = search

#query loop performs search for a hotel within 5000 meters of each city from hotel_df
for city in range(len(hotel_df["City"])):
    city_lat = hotel_df.loc[city]["Latitude"]
    city_long = hotel_df.loc[city]["Longitude"]
    coord = f"{city_lat},{city_long}"

    Query_parameters = {
        "location": coord,
        "radius": radius,
        "type": query_type,
        "key": g_key
    }

    Gmap_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    query_request = requests.get(Gmap_url, params=Query_parameters)
    query_response = query_request.json()
    try:
        hotel_list.append(query_response["results"][0]["name"])
    except IndexError:
        hotel_list.append("Not Found")

In [21]:
#testing if query was successful
print(hotel_list)

['Moon light Furnished Units', 'سالن پیرایش امید', 'Roy Mansion 3', 'Mian Lodge', 'Sur Plaza Hotel', 'Mr Bo Bo', 'Casa Dora, Bubaque', 'کافه گیم استورم', 'Govt Guest House (Dac Banglo)', 'Hotel JM Puerto del Rosario', 'HAMDAN PLAZA HOTEL SALALAH', 'Not Found', 'Ravena Cassino Hotel', 'B&B La Casona de Nazaret', 'The Chancellor on Currie']


In [24]:
#convert query results to dataframe, outer merge on hotel_df
list_df = pd.DataFrame(hotel_list,columns=["Hotel"])
df_merged = hotel_df.merge(list_df, how='outer', left_index=True, right_index=True)

hotel_df

,City,Country,Max Temperature,Windspeed,Cloudiness,Humidity,Latitude,Longitude,Hotel
0,umm lajj,SA,79.07,7.58,0.0,51.0,25.0213,37.2685,Moon light Furnished Units
1,dezful,IR,78.57,4.79,0.0,22.0,32.3811,48.4058,سالن پیرایش امید
2,chandrakona,IN,73.81,5.79,0.0,22.0,22.7333,87.5167,Roy Mansion 3
3,mailsi,PK,74.62,5.39,0.0,19.0,29.8003,72.1758,Mian Lodge
4,sur,OM,75.63,3.80,0.0,52.0,22.5667,59.5289,Sur Plaza Hotel
5,pyu,MM,73.76,3.98,0.0,28.0,18.4833,96.4333,Mr Bo Bo
6,bubaque,GW,77.49,3.56,0.0,50.0,11.2833,-15.8333,"Casa Dora, Bubaque"
7,dehloran,IR,76.69,2.80,0.0,18.0,32.6941,47.2679,کافه گیم استورم
8,bheramara,BD,72.45,3.29,0.0,22.0,24.0232,88.9925,Govt Guest House (Dac Banglo)
9,puerto del rosario,ES,71.60,6.91,0.0,46.0,28.5004,-13.8627,Hotel JM Puerto del Rosario


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [33]:
#creates pins for heatmap and overlays them
pins = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(pins)
fig

Figure(layout=FigureLayout(height='420px'))